In [ ]:
%load_ext autoreload
%autoreload 2

!pip install pandas

import random
import pandas as pd

import helper.Counters as Counters
import helper.Inputs as Inputs
import helper.util as util
import helper.Powersort as Powersort
import helper.Timsort as Timsort

import helper.Books as Books

## Helper Functions

In [26]:
def cost(lst, sorter):
    wrapped = [Counters.ComparisonCounter(x) for x in lst]
    Counters.reset_counters()
    sorter.sort(wrapped)
    assert Counters.ComparisonCounter.EQ_COMPARISONS == 0
    
    return {
        'Algorithm': sorter.name(),
        'Comparisons': Counters.ComparisonCounter.COMPARISONS,
        'Mergecost': Counters.MergeCosts.MERGECOST
    }

In [27]:
def compare_sorters(lst, sorters=(Powersort, Timsort)):
    sorters = sorted(sorters, key = lambda sorter: sorter.name())
    
    df = pd.DataFrame([cost(lst, sorter) for sorter in sorters])
    # Force columns to use type object to allow for mixed types (renders nicer)
    column_types = {'Algorithm': 'str', 'Comparisons': 'object', 'Mergecost': 'object'}
    df = df.astype(column_types)
    df = df.append({
        'Algorithm': 'improvement (%)', 
        'Comparisons': ((df['Comparisons'][1] - df['Comparisons'][0]) / df['Comparisons'][1]) * 100, 
        'Mergecost': ((df['Mergecost'][1] - df['Mergecost'][0]) / df['Mergecost'][1]) * 100,
    }, ignore_index=True)

    return df

## Random Permutation

The simplest way of generating an input list, without much thought, for the competition is to generate a set of random permutation.

In [28]:
X = Inputs.random_permutation(10000)
name = "RandomPermutation"

compare_sorters(X)

,Algorithm,Comparisons,Mergecost
0,powersort,119849,79760
1,timsort,119930,79840
2,improvement (%),0.0675394,0.1002


## Bible.txt


A more intuitive way is to base your input list on some real-world data. For this example, the list consists of words from the bible. Note that this is valid as alphabetical letters have inherent lexicographic ordering.

In [29]:
X = Books.list_of_words_bible()
name = "Bible"

compare_sorters(X)

,Algorithm,Comparisons,Mergecost
0,powersort,10001936,10721664
1,timsort,10004865,10722878
2,improvement (%),0.0292758,0.0113216


## Mixed long and short runs

In [30]:
def input_generator(n, rng):
    p = 0.8
    short = (1, 100)
    long = (1000, 10000)
    num_runs = max(2, int(n / (p * 0.5 * sum(short) + (1-p) * 0.5 * sum(long))))
    run_lens = Inputs.tims_random_run_lengths(num_runs, p, short, long, rng)
    nn = sum(run_lens)
    lst = [0] * nn
    
    Inputs.fill_with_asc_runs_same(lst, run_lens, 1, use_n_as_last_entry=False)
    lst = util.rank_reduce_ties_desc(lst)
    
    return lst

In [41]:
X = input_generator(200000, random.Random(437895634))
name = "MixedLongShort"

compare_sorters(X)

,Algorithm,Comparisons,Mergecost
0,powersort,2534463,2399314
1,timsort,2535470,2399760
2,improvement (%),0.0397165,0.0185852


## Random unary strings

In [32]:
def input_generator(n, rng):
    u = int(n ** 0.5)
    
    return Inputs.random_uary_array(u, n, rng)

In [50]:
X = input_generator(1000, random.Random(34253563))
name = "RandomUnary"

compare_sorters(X)

,Algorithm,Comparisons,Mergecost
0,powersort,8085,4000
1,timsort,8085,4000
2,improvement (%),0,0


## Extra (Helper) Function: save most recently generated list to filesystem

In [51]:
print("Saving array as", name + '.txt')

with open(name + '.txt', "w") as f:
    f.write(str(X))
    f.close()

Saving array as RandomUnary.txt
